# Test Driven Development of Character class layout

This notebook serves two purposes:
- To develop the program (layout of classes, data structures, etc) for the interactive character sheet
- To provide an example of test driven development

## Objective

We want to be able to initialise/create a Character object that can return all of the essential information about about the D&D Character. It should also be able to level up and contain methods for all actions in combat.

## List of Requirements (Update this over time)
1. Initialise a `Character` object
2. Initialiase `Character` with some details
    - A string for each of: `name`, `race`, and `character_class`
    - An integer for `level`, which defaults to 1
3. Make it that `Character` "has a" `Race` and "has a" `CharacterClass`
    - Each is a class of it's own
    - Build in the ability to check the string with a `__str__` method
4. Further develop the `Race` class
    - It has subclasses for each race from D&D 5e
    - The `__str__` method returns the race name
5. Further develop the `CharacterClass` class
    - It has subclasses for each character class from D&D 5e
    - The `__str__` method returns the character class name
6. Initialise a `Player` class which "is a" `Character`
7. Initialise a `NonPlayer` class which "is a" `Character`
8. Initialise a `Weapon` class
9. Initialise a `Spell` class
10. Initialise an `Ability` class
    - `Ability` has the `check()` method and `modifier` and `score` properties
11. Each ability (`Strength` etc...) is a subclass of `Ability`
12. Each `Ability` subclass has skills
    - Skills and saving throws are kept in a dictionary called `proficiencies`
    - `proficiencies` key:value pairs are all `skill : <bool>`
13. Initialise a `Background` class
   - Contains a list of `proficiencies` and a description string
14. Each `Character` has a list of `Abilities`
   - The abilities for each character have scores and proficiencies
   - The proficiencies for each character are determined by the `Race`, `CharacterClass` and `Background`
   - `Race` and `CharacterClass` each have a list of `proficiencies` (or a list to choose from), just like in `Background`
15. Include function to create a custom `Race` and a `CharacterClass`
16. 

#### First, import the testing modules:

In [1]:
# Set the file name for unit testing iwth ipytest
__file__ = "character_scripting.ipynb"

import pytest
import ipytest.magics

## 1. Initialise a `Character` object

#### Make sure the tests are defined *before* the main code is written:

In [7]:
%%run_pytest -v --tb=line

def test_Character_can_be_created():
    assert Character()

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 1 item

character_scripting.py::test_Character_can_be_created <- <ipython-input-7-24813ad9a017> FAILED [100%]

================================== FAILURES ===================================
<ipython-input-7-24813ad9a017>:3: NameError: name 'Character' is not defined
========================== 1 failed in 0.10 seconds ===========================


#### Test failed (because we are yet to define `Character`). "Refactor" the code:

In [8]:
%%run_pytest

class Character(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 1 item

character_scripting.py .                                                 [100%]

========================== 1 passed in 0.06 seconds ===========================


### [This process](https://en.wikipedia.org/wiki/Test-driven_development#Test-driven_development_cycle) should be repeated for each new requirement
1. **Write** the test(s)
```python
def test_<test_description>():
    # Some code (potentially)
    assert true_statement
```
2. **Run** the tests
```python
%%run_pytest -v --tb=line
```
3. **Refactor** (update or write new code - including tests) until all tests pass
4. **Repeat** for the next requirement

## 2. Initialiase `Character` with some details
- A string for each of: `name`, `race`, and `character_class`
- An integer for `level`, which defaults to 1

1. **Write** the test(s)

In [9]:
%%run_pytest -v --tb=line

def test_Character_fields():

    field_list = ["name", "_race", "character_class", "level"]
    inputs = "Merret","Halfling","Ranger", 8
    test_character = Character(*inputs) # * operator unpacks tuple (** for dict)
    test_fields = [field for field in dir(test_character)]
    
    for inpt, field in zip(inputs, field_list):
        if field not in test_fields:
            assert getattr(test_character, field)
        elif inpt != getattr(test_character,field):
            assert inpt == getattr(test_character,field)
    assert True

def test_Character_for_level_default():
    assert 1 == Character("Merret","Halfling","Ranger").level

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 3 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-7-24813ad9a017> PASSED [ 33%]
character_scripting.py::test_Character_fields <- <ipython-input-9-72fcd976dc44> FAILED [ 66%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-9-72fcd976dc44> FAILED [100%]

================================== FAILURES ===================================
<ipython-input-9-72fcd976dc44>:6: TypeError: object() takes no parameters
<ipython-input-9-72fcd976dc44>:17: TypeError: object() takes no parameters
===================== 2 failed, 1 passed in 0.17 seconds ======================


2. **Run** the tests

In [10]:
%%run_pytest -v --tb=line

class Character(object):
    
    def __init__(self, name, race, character_class, level=1):
        self.name = name
        self._race = race
        self.character_class = character_class
        self.level = level

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 3 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-7-24813ad9a017> FAILED [ 33%]
character_scripting.py::test_Character_fields <- <ipython-input-9-72fcd976dc44> PASSED [ 66%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-9-72fcd976dc44> PASSED [100%]

================================== FAILURES ===================================
<ipython-input-7-24813ad9a017>:3: TypeError: __init__() missing 3 required positional arguments: 'name', 'race', and 'character_class'
===================== 1 failed, 2 passed in 0.15 seconds ======================


3. **Refactor** - The initial test does not include any inputs to `Character`. Include default values.

In [11]:
%%run_pytest -v --tb=line

class Character(object):
    
    def __init__(self, name="Merret", race="Halfling",
                 character_class="Ranger", level=1):
        self.name = name
        self._race = race
        self.character_class = character_class
        self.level = level

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 3 items

character_scripting.py::test_Character_can_be_created <- <ipython-input-7-24813ad9a017> PASSED [ 33%]
character_scripting.py::test_Character_fields <- <ipython-input-9-72fcd976dc44> PASSED [ 66%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-9-72fcd976dc44> PASSED [100%]

========================== 3 passed in 0.12 seconds ===========================


## 3. Make it that `Character` "has a" `Race` and "has a" `CharacterClass`
- Each is a class of it's own
- Build in the ability to check the string with a `__str__` method

In [12]:
def test_Character_has_a_Race():
    assert isinstance(Character()._race, Race)
def test_Character_has_a_CharacterClass():
    assert isinstance(Character().character_class, CharacterClass)

In [13]:
%%run_pytest -v --tb=line

class Race(object):
    
    def __init__(self, race):
        self._race = race
    
    def __str__(self):
        return self._race


class CharacterClass(object):
    
    def __init__(self, character_class):
        self.character_class = character_class
    
    def __str__(self):
        return self.character_class


class Character(object):
    
    def __init__(self, name="Merret", race="Halfling",
                 character_class="Ranger", level=1):
        self.name = name
        self._race = Race(race) # Changed Line
        self.character_class = CharacterClass(character_class) # Changed Line
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 5 items

character_scripting.py::test_Character_has_a_Race <- <ipython-input-12-58755c1bd107> PASSED [ 20%]
character_scripting.py::test_Character_has_a_CharacterClass <- <ipython-input-12-58755c1bd107> PASSED [ 40%]
character_scripting.py::test_Character_can_be_created <- <ipython-input-7-24813ad9a017> PASSED [ 60%]
character_scripting.py::test_Character_fields <- <ipython-input-9-72fcd976dc44> FAILED [ 80%]
character_scripting.py::test_Character_for_level_default <- <ipython-input-9-72fcd976dc44> PASSED [100%]

================================== FAILURES ===================================
<ipython-input-9-72fcd976dc44>:13: AssertionError: assert 'Halfling' == <__main_

#### The new requirement breaks the `test_Character_fields` test. Refactor the tests to make use of the `__str__` method:

In [14]:
%%run_pytest

def test_Character_fields():

    field_list = ["name", "_race", "character_class", "level"]
    inputs = "Merret","Halfling","Ranger", 8
    test_character = Character(*inputs) # * operator unpacks tuple (** for dict)
    test_fields = [field for field in dir(test_character)]
    
    for inpt, field in zip(inputs, field_list):
        if field not in test_fields:
            assert getattr(test_character, field)
        # Refactor here:
        elif str(inpt) != str(getattr(test_character,field)):
            assert str(inpt) == str(getattr(test_character,field))
    assert True

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 5 items

character_scripting.py .....                                             [100%]

========================== 5 passed in 0.09 seconds ===========================


## 4. Further develop the `Race` class
   - It has subclasses for each race from D&D 5e
   - The `__str__` method returns the race name

In [89]:
# Is a Race
def test_Race_subclasses():

    race_list = [
        "Dwarf", "Elf", "Halfling", "Human", "Dragonborn",
        "Gnome", "HalfElf", "HalfOrc", "Tiefling"
    ]

    subclasses = {cls.__name__ : cls() for cls in Race.__subclasses__()}

    for race in race_list:
        if race not in subclasses:
            assert False, '{0} is not a subclass of Race'.format(race)
        elif str(subclasses[race]) != race:
            assert False, '{0} __str__ method is incorrect'.format(race)
    assert True

In [91]:
%%run_pytest -v --tb=line

class Race(object):
    
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__ # returns the name of the class

    
class Dwarf(Race):
    pass
class Elf(Race):
    pass
class Halfling(Race):
    pass
class Human(Race):
    pass
class Dragonborn(Race):
    pass
class Gnome(Race):
    pass
class HalfElf(Race):
    pass
class HalfOrc(Race):
    pass
class Tiefling(Race):
    pass


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 14 items

character_scripting.py ...FF.........                                    [100%]

================================== FAILURES ===================================
_________________________ test_Player_is_a_Character __________________________

    def test_Player_is_a_Character():
>       assert isinstance(Player(),Character)
E       AssertionError

<ipython-input-20-0e410eeb099a>:2: AssertionError
________________________ test_NonPlayer_is_a_Character ________________________

    def test_NonPlayer_is_a_Character():
>       assert isinstance(NonPlayer(),Character)
E       AssertionError

<ipython-input-22-ec70e452f4ca>:2: AssertionError
===================== 2 failed, 12 passed in 0.26 seconds =====================


#### Must update assignment of `_race` in `Character`

In [17]:
%%run_pytest

class Character(object):
    
    def __init__(self, name="Merret", race="Halfling",
                 character_class="Ranger", level=1):
        self.name = name
        self._race = globals()[race.title()]() # Changed Line
        self.character_class = CharacterClass(character_class)
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 6 items

character_scripting.py ......                                            [100%]

========================== 6 passed in 0.09 seconds ===========================


## 5. Further develop the `CharacterClass` class
   - It has subclasses for each character class from D&D 5e
   - The `__str__` method returns the character class name
   - Each subclass has a flag for if it is a spellcaster

In [105]:
def test_CharacterClass_subclasses():

    class_list = [
        'Barbarian', 'Bard', 'Cleric', 'Druid', 'Fighter', 'Monk',
        'Paladin', 'Ranger', 'Rogue', 'Sorcerer', 'Warlock', 'Wizard'
    ]

    subclasses = {cls.__name__ : cls() for cls in CharacterClass.__subclasses__()}

    for cls in class_list:
        # Is a Character Class
        if cls not in subclasses:
            assert False, '{0} is not a subclass of CharacterClass'.format(cls)
        # __str__ method works properly
        elif str(subclasses[cls]) != cls:
            assert False, '{0} __str__ method is incorrect'.format(cls)
        # Has spellcaster boolean
        elif not isinstance(subclasses[cls].spellcaster,bool):
            assert False, '{0}.spellcaster should be boolean'.format(cls)
    assert True

In [86]:
%%run_pytest

class CharacterClass(object):
    
    def __init__(self, spellcaster=False):
        self.spellcaster = spellcaster
    
    def __str__(self):
        return type(self).__name__ # returns the name of the class

    
class Barbarian(CharacterClass):
    def __init__(self):
        super().__init__()

class Bard(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Cleric(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Druid(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Fighter(CharacterClass):
    def __init__(self):
        super().__init__()

class Monk(CharacterClass):
    def __init__(self):
        super().__init__()

class Paladin(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Ranger(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Rogue(CharacterClass):
    def __init__(self):
        super().__init__()

class Sorcerer(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Warlock(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)

class Wizard(CharacterClass):
    def __init__(self):
        super().__init__(spellcaster=True)


# Same problem will occur as when changing Race, so update assignment of
# character_class in Character now
class Character(object):
    
    def __init__(self, name="Merret", race="Halfling",
                 character_class="Ranger", level=1):
        self.name = name
        self._race = globals()[race.title()]()
        self.character_class = globals()[character_class.title()]() # Changed
        self.level = level


============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 14 items

character_scripting.py ....FF........                                    [100%]

================================== FAILURES ===================================
_________________________ test_Player_is_a_Character __________________________

    def test_Player_is_a_Character():
>       assert isinstance(Player(),Character)
E       AssertionError

<ipython-input-20-0e410eeb099a>:2: AssertionError
________________________ test_NonPlayer_is_a_Character ________________________

    def test_NonPlayer_is_a_Character():
>       assert isinstance(NonPlayer(),Character)
E       AssertionError

<ipython-input-22-ec70e452f4ca>:2: AssertionError
===================== 2 failed, 12 passed in 0.25 seconds =====================


## 6. Initialise a `Player` class which "is a" `Character`

In [92]:
def test_Player_is_a_Character():
    assert isinstance(Player(),Character)

In [93]:
%%run_pytest

class Player(Character):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 14 items

character_scripting.py ...F..........                                    [100%]

================================== FAILURES ===================================
________________________ test_NonPlayer_is_a_Character ________________________

    def test_NonPlayer_is_a_Character():
>       assert isinstance(NonPlayer(),Character)
E       AssertionError

<ipython-input-22-ec70e452f4ca>:2: AssertionError
===================== 1 failed, 13 passed in 0.23 seconds =====================


## 7. Initialise a `NonPlayer` class which "is a" `Character`

In [22]:
def test_NonPlayer_is_a_Character():
    assert isinstance(NonPlayer(),Character)

In [94]:
%%run_pytest

class NonPlayer(Character):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 14 items

character_scripting.py ..............                                    [100%]

========================== 14 passed in 0.15 seconds ==========================


## 8. Initialise a `Weapon` class

In [24]:
def test_Weapon_can_be_created():
    assert Weapon()

In [25]:
%%run_pytest

class Weapon(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 11 items

character_scripting.py ...........                                       [100%]

========================== 11 passed in 0.13 seconds ==========================


## 9. Initialise a `Spell` class

In [26]:
def test_Spell_can_be_created():
    assert Spell()

In [27]:
%%run_pytest

class Spell(object):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 12 items

character_scripting.py ............                                      [100%]

========================== 12 passed in 0.13 seconds ==========================


## 10. Initialise an `Ability` class
   - `Ability` has the `check()` method and `modifier` and `score` properties

In [95]:
def test_Ability_fields():

    field_list = ["check", "modifier", "score"]
    test_fields = [field for field in dir(Ability())]
    
    for field in field_list:
        if field not in test_fields:
            assert isinstance(getattr(Ability(),field),int)
    assert True

In [96]:
%%run_pytest -v --tb=line

import random

class Ability(object):
    
    def __init__(self, score=10):
        self.score = score
        self.modifier = int((score - 10) / 2)
        
    def check(self):
        return random.randint(1,20) + self.modifier
    

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1 -- c:\users\dal189\documents\dnd5e-python\env\scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collecting ... collected 14 items

character_scripting.py::test_Character_has_a_Race <- <ipython-input-12-58755c1bd107> PASSED [  7%]
character_scripting.py::test_Character_has_a_CharacterClass <- <ipython-input-12-58755c1bd107> PASSED [ 14%]
character_scripting.py::test_Character_fields <- <ipython-input-14-679f278e2370> PASSED [ 21%]
character_scripting.py::test_NonPlayer_is_a_Character <- <ipython-input-22-ec70e452f4ca> PASSED [ 28%]
character_scripting.py::test_Weapon_can_be_created <- <ipython-input-24-dcf591d50261> PASSED [ 35%]
character_scripting.py::test_Spell_can_be_created <- <ipython-input-26-0a16777caab3> PASSED [ 42%]
character_scripting.py::test_Ability_subclasses <- <ipython-inpu

## 11. Each ability (`Strength` etc...) is a subclass of `Ability`

In [102]:
# Is an Ability
def test_Ability_subclasses():

    ability_list = [
        "Strength", "Dexterity", "Constitution",
        "Intelligence", "Wisdom", "Charisma"
    ]

    subclasses = {cls.__name__ : cls for cls in Ability.__subclasses__()}
    
    for ability in ability_list:
        if ability not in subclasses:
            assert False, '{0} is not a subclass of Ability'.format(ability)
    assert True

In [103]:
%%run_pytest

class Strength(Ability):
    pass
class Dexterity(Ability):
    pass
class Constitution(Ability):
    pass
class Intelligence(Ability):
    pass
class Wisdom(Ability):
    pass
class Charisma(Ability):
    pass

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 14 items

character_scripting.py ..............                                    [100%]

========================== 14 passed in 0.15 seconds ==========================


## 12. Each `Ability` subclass has skills
   - Skills and saving throws are kept in a dictionary called `proficiencies`
   - `proficiencies` key:value pairs are all `skill : <bool>`

In [154]:
# Refactor the previous test
def test_Ability_subclasses():

    ability_list = [
        "Strength", "Dexterity", "Constitution",
        "Intelligence", "Wisdom", "Charisma"
    ]

    subclasses = {cls.__name__ : cls() for cls in Ability.__subclasses__()}
    
    for ability in ability_list:
        # Is an Ability
        if ability not in subclasses:
            assert False, '{0} is not a subclass of Ability'.format(ability)
        # Has proficiencies dictionary
        elif not isinstance(subclasses[ability].proficiencies,dict):
            assert False, '{0} does not have proficiencies dict'.format(ability)
        # Values in proficiencies are booleans
        profs = subclasses[ability].proficiencies
        for prof in profs:
            if not isinstance(profs[prof],bool):
                assert False, '{0}.proficiencies[{1}] is not bool'.format(
                    ability, prof)
    assert True

In [155]:
%%run_pytest

class Strength(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throws", "Athletics"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Dexterity(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = [
            "Saving Throws", "Acrobatics",
            "Sleight of Hand", "Stealth"
        ]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Constitution(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = ["Saving Throws"]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Intelligence(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = [
            "Saving Throws", "Arcana", "History",
            "Investigation", "Nature", "Religion"
        ]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Wisdom(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = [
            "Saving Throws", "Animal Handling", "Insight",
            "Medicine", "Perception", "Survival"
        ]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    
class Charisma(Ability):
    
    def __init__(self, score=10, proficiencies=[]):
        skill_list = [
            "Saving Throws", "Deception", "Intimidation",
            "Performance", "Persuasion"
        ]
        proficiencies = [prof.title() for prof in proficiencies]
        self.proficiencies = {
            skill:skill in proficiencies for skill in skill_list
        }
        super().__init__(score)
    

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.7.1, py-1.5.4, pluggy-0.7.1
rootdir: C:\Users\dal189\Documents\dnd5e-python, inifile:
collected 13 items

character_scripting.py .............                                     [100%]

========================== 13 passed in 0.15 seconds ==========================


## 13. Initialise a `Background` class
   - Contains a list of `proficiencies` and a description string

## 14. Each `Character` has a list of `Abilities`
   - The abilities for each character have scores and proficiencies
   - The proficiencies for each character are determined by the `Race`, `CharacterClass` and `Background`
   - `Race` and `CharacterClass` each have a list of `proficiencies` (or a list to choose from), just like in `Background`

In [ ]:
class Character(object):
    
    def __init__(self, name="Merret", race="Halfling",
                 character_class="Ranger", level=1):
        self.name = name
        self._race = globals()[race.title()]()
        self.character_class = globals()[character_class.title()]()
        self.level = level


## 15. Include function to create a custom `Race` and a `CharacterClass`

## Convert Notebook to a Python Script

In [ ]:
# Main class for Player Characters - "is a" Character
class Player(Character):
    
    def __init__(self, name, race, character_class, ability_scores, level=1):
        super().__init__(name, race, character_class, level)
        self.__spellcaster = str(self._class) in [
            "Bard", "Druid", "Ranger", "Sorceror", "Wizard", "Warlock"]
        self.ability_scores = {
            "Strength" : ability_scores[0],
            "Dexterity" : ability_scores[1],
            "Constitution" : ability_scores[2],
            "Intelligence" : ability_scores[3],
            "Wisdom" : ability_scores[4],
            "Charisma" : ability_scores[5]
        }
        
    def __str__(self):
        return "A level {0} {1} {2} called {3}".format(
            self.level, 
            str(self._race).title(),
            str(self._class),
            self.name.title())
    
    def attack(self):
        pass

In [ ]:
# Main class for Non-Player Characters - "is a" Character
class NonPlayer(Character):
    pass

In [ ]:
merret = Player("Merret Strongheart",
                   "Halfling",
                   "ranger",
                   [12, 20, 12, 8, 16, 8],
                   9)

In [ ]:
print(merret._class)

In [ ]:
merret._Player__spellcaster

In [ ]:
print(merret)

In [ ]:
despair = Player("Despair", "Tiefling", "Sorceror", [9, 11, 15, 14, 13, 20], 8)

In [ ]:
despair._Player__spellcaster

In [ ]:
print(despair)